# This is the source code used in Ryerson-Intel project
Author: Alice

Created: Dec 2019

Modified for EMD analysis: Mar 2, 2020

Classification for EMD fixed frame

Using the EMD features and its delta features

In [29]:
# Mount the google drive to get the feature files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Import system packages
import os
import glob

# Importing packages
import pandas as pd
import numpy as np
from numpy import set_printoptions

# Preparing dataset
from sklearn.preprocessing import MinMaxScaler
import random

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB



# Cross Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Performance Measure
from sklearn import metrics
import statistics

# Plotting
import matplotlib.pyplot as plt

In [0]:
# Control setting to see which types of data
FCTOL = 'FC15Per' #FC10Per or FC15Per
SEG   = 'Triad'   #FixedFrameSize, Triad, or '' 

## Preparing the Dataset
Loading the feature csv file into arrays and normalized the features to range of [0,1]. There are 49 signals in total with 21 stress signals and 28 non-stress signals

In [32]:
# Setting folder directory
datapath = '/content/drive/My Drive/2019 DDK Paper/Journal/Features/EMD/GITA Features/'
# use the folder name for segmentation: '16K Fixed Frame Size', '16K Triad', '4k', and 'EMDDeltaEMD'
segmentation = 'EMDDeltaEMD'  
# The actual datafile is in Statistical Summary folder
datapath = datapath + segmentation +'/'

filename = SEG+'*'+FCTOL+'*.csv'
#addrs = glob.glob(datapath+'*.csv')
addrs = glob.glob(datapath+'*'+filename)

print(segmentation)
print(os.path.basename(addrs[0]))
print(os.path.basename(addrs[1]))



EMDDeltaEMD
EMDHCTriadFC15Per.csv
EMDPDTriadFC15Per.csv


In [0]:
# Construct dataframe for analysis

# Importing data matrices and remove rows containing nan
HCaddr = [addr for addr in addrs if 'HC' in addr ]
PDaddr = [addr for addr in addrs if 'PD' in addr ]

dfHCOriginal = pd.read_csv(HCaddr[0])
dfHCOriginal.dropna()
print('HC matrix dim: ', dfHCOriginal.shape)

dfPDOriginal = pd.read_csv(PDaddr[0])
dfPDOriginal.dropna()
print('PD matrix dim: ',dfPDOriginal.shape)




# Remove rows that contain 0 for all std features
def removeZeroData(dftmp):
  colnames = dftmp.columns
  stdcol = [header for header in colnames if 'Std' in header]
  stdtotal = dftmp[stdcol].sum(axis=1)
  dropIdx = [i for i,std in enumerate(stdtotal) if std==0]
  #print(stdtotal)
  print('Removing ', len(dropIdx), ' rows')
  for i in dropIdx:
    dftmp = dftmp.drop(dftmp.index[i])
  return dftmp

dfPDOriginal = removeZeroData(dfPDOriginal)
pdrow, pdcol = dfPDOriginal.shape
print(dfPDOriginal.shape)

dfHCOriginal = removeZeroData(dfHCOriginal)
hcrow, hccol = dfHCOriginal.shape
print(dfHCOriginal.shape)



# Combining the HC and PD dataframes
dfHCPD = pd.concat([dfHCOriginal,dfPDOriginal], axis=0)
hclabels = np.ones(hcrow, dtype=int)
pdlabels = np.zeros(pdrow, dtype=int)
                   
Labels = np.reshape(np.append(hclabels, pdlabels), (-1,1)) # Label for HC=1 and PD=0
print(hcrow,pdrow)
print('Label size', Labels.shape)

# features
colnames = dfHCPD.columns
hcpdfeatures = np.array(dfHCPD.iloc[:,1:].values)

print('Features: ', colnames)
print('dfHCPD matrix dim: ', dfHCPD.shape)
print('size of HCPD features: ', hcpdfeatures.shape)



HC matrix dim:  (50, 434)
PD matrix dim:  (50, 434)
Removing  0  rows
(50, 434)
Removing  0  rows
(50, 434)
50 50
Label size (100, 1)
Features:  Index(['subject', 'segmentNum', 'numIMFMean', 'OBW1Mean', 'OBW2Mean',
       'OBW3Mean', 'OBW4Mean', 'OBW5Mean', 'OBW6Mean', 'OBW7Mean',
       ...
       'deltaFCenter1Kurt', 'deltaFCenter2Kurt', 'deltaFCenter3Kurt',
       'deltaFCenter4Kurt', 'deltaFCenter5Kurt', 'deltaFCenter6Kurt',
       'deltaFCenter7Kurt', 'deltaFCenter8Kurt', 'deltaAmpKurt',
       'deltaDurKurt'],
      dtype='object', length=434)
dfHCPD matrix dim:  (100, 434)
size of HCPD features:  (100, 433)


In [0]:
# Normalizing the features into [0,1]
scaler = MinMaxScaler(feature_range = (0,1)) # scale the values to min 0, max 1
rescaledfeat = np.array(scaler.fit_transform(hcpdfeatures)) # fit the trainning feature X into scaler

set_printoptions(precision=3) # how many decimal places.
print(rescaledfeat[0:5,:])
print(rescaledfeat.shape)



[[0.278 0.429 0.499 ... 0.881 0.193 0.447]
 [0.667 0.467 0.592 ... 0.607 0.379 0.649]
 [0.333 0.567 0.    ... 0.399 0.381 0.107]
 [0.167 0.573 0.63  ... 0.391 0.095 0.127]
 [0.5   0.418 0.877 ... 0.598 0.196 0.433]]
(100, 433)


## Classification 
The classifiers tested in this project include Decision Tree, Each classifier uses 5-fold for validation.

In [0]:
# Classifiers
# Decision Tree
DT = DecisionTreeClassifier(random_state=0)
# Gradient Boost
GB = GradientBoostingClassifier(n_estimators=100, 
                                learning_rate=1.0, 
                                max_depth=1, 
                                random_state=0)

# Support Vector Machine
SVM = SVC(random_state = 0, gamma='scale',probability = True)

# Linear Discrimant Analysis
LDA = LinearDiscriminantAnalysis()

# Gaussian Naive Bayes
GNB = GaussianNB()



In [0]:
# Classificaiton function with K-Fold cross-validation with user define iterations to get the standard deviation 

def classify(model, Iterations, num_folds, rescaledX, y):
    acc = []
    pre = []
    rec = []
    rocauc = []
    f1 = []
    Skip = False

    bestAcc = 0
    beststd = 0



    for i in range(Iterations):

        # Shuffle the dataset for each iteration
        data = list(zip(rescaledX,y))
        random.shuffle(data)
        rescaledX, y = zip(*data)
        rescaledX = np.array(rescaledX)
        y = np.array(y)
        #y = y.reshape(-1,1)

        # Perform 5-fold validation
        kfold = KFold(n_splits = num_folds)#, shuffle=True, random_state = None)
        results = cross_val_score(model, rescaledX, y.ravel(), cv = kfold)
        if results.mean()>bestAcc:
          bestAcc = results.mean()
          beststd = results.std()
          print(results)

        for train_index, test_index in kfold.split(rescaledX):
            X_train, X_test = rescaledX[train_index], rescaledX[test_index]
            y_train, y_test = y[train_index], y[test_index]

            y_total = sum(y_test)
            if (y_total == 0) or (y_total == len(y_test)):
                Skip = True
                
            if not Skip:
                # perform training and testing
                if model == SVC:
                    model.fit(X_train,y_train)
                else:
                    model.fit(X_train,y_train.ravel())
                #dtscores = DT.score(X_test,y_test)
                yPred = model.predict(X_test)


                # record performance
                acc = np.append(acc,metrics.accuracy_score(y_test, yPred)) 
                pre = np.append(pre,metrics.precision_score(y_test,
                                                            yPred, 
                                                            pos_label=1, 
                                                            average='macro', 
                                                            labels=np.unique(yPred)))
                rec = np.append(rec,metrics.recall_score(y_test,
                                                         yPred, 
                                                         pos_label=1, 
                                                         average='macro', 
                                                         labels=np.unique(yPred)))
                rocauc = np.append(rocauc, metrics.roc_auc_score(y_test, 
                                                                 yPred,
                                                                 average='macro'))
                f1 = np.append(f1, metrics.f1_score(y_test,yPred))
    return acc, pre, rec, rocauc, f1, bestAcc, beststd

    

### Parameter setting
The classification are set to 100 iteration using 5-fold cross validation.

NOTE: You might encounter single predicted labels instead of 2 class labels. This will return an error. The easiest work around is to run the cell again to shuffle tthe data. 

In [0]:
Iterations = 200
num_folds = 10
seed = 15

## Support Vector Machine

In [0]:
print('Support Vector Machine')
acc, pre, rec, rocauc, f1,bestacc, beststd = classify(SVM, Iterations, num_folds,  rescaledfeat, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))
print('best iteration accuracy: ', bestacc, beststd)


Support Vector Machine
[0.7 0.7 0.8 0.7 0.9 0.7 0.7 0.7 0.8 0.7]
[0.7 0.6 0.7 0.8 0.8 0.7 0.7 0.8 0.8 1. ]
[0.8 0.9 0.5 0.8 0.9 0.8 0.8 0.7 0.8 0.7]
Accuracy (mean, std): 0.7192307692307692 0.1328192363887707
Precision (mean, std): 0.7492280689396074 0.13153728110000723
Recall (mean, std): 0.7304293462947309 0.13153728110000723
Area under the Receiver Operating Characteristic Curve 0.7289500563539025 0.1300009173205882
F1-score: 0.7380680067505685 0.14305958476579367
best iteration accuracy:  0.77 0.11000000000000001


In [0]:
# Classificaiton function with K-Fold cross-validation with user define iterations to get the standard deviation 

def findbestresults(model, Iterations, num_folds, rescaledX, y):
    
    Skip = False

    bestAcc = 0

    for i in range(Iterations):

        # Shuffle the dataset for each iteration
        data = list(zip(rescaledX,y))
        random.shuffle(data)
        rescaledX, y = zip(*data)
        rescaledX = np.array(rescaledX)
        y = np.array(y)
        #y = y.reshape(-1,1)

        # Perform 5-fold validation
        kfold = KFold(n_splits = num_folds)#, shuffle=True, random_state = None)
        results = cross_val_score(model, rescaledX, y.ravel(), cv = kfold)
        if results.mean()>bestAcc:
          bestAcc = results.mean()
          
          print(results)

          acc = []
          pre = []
          rec = []
          rocauc = []
          f1 = []

          for train_index, test_index in kfold.split(rescaledX):
              X_train, X_test = rescaledX[train_index], rescaledX[test_index]
              y_train, y_test = y[train_index], y[test_index]

              y_total = sum(y_test)
              if (y_total == 0) or (y_total == len(y_test)):
                  Skip = True
                  
              if not Skip:
                  # perform training and testing
                  if model == SVC:
                      model.fit(X_train,y_train)
                  else:
                      model.fit(X_train,y_train.ravel())
                  #dtscores = DT.score(X_test,y_test)
                  yPred = model.predict(X_test)


                  # record performance
                  acc = np.append(acc,metrics.accuracy_score(y_test, yPred)) 
                  pre = np.append(pre,metrics.precision_score(y_test,
                                                              yPred, 
                                                              pos_label=1, 
                                                              average='macro', 
                                                              labels=np.unique(yPred)))
                  rec = np.append(rec,metrics.recall_score(y_test,
                                                          yPred, 
                                                          pos_label=1, 
                                                          average='macro', 
                                                          labels=np.unique(yPred)))
                  rocauc = np.append(rocauc, metrics.roc_auc_score(y_test, 
                                                                  yPred,
                                                                  average='macro'))
                  f1 = np.append(f1, metrics.f1_score(y_test,yPred))
    return acc, pre, rec, rocauc, f1

In [0]:
Iterations = 200
num_folds = 10


print('Support Vector Machine')
acc, pre, rec, rocauc, f1 = findbestresults(SVM, Iterations, num_folds,  rescaledfeat, Labels)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))



Support Vector Machine
[0.8 0.9 0.9 0.9 0.6 0.6 0.7 0.6 0.4 0.7]
[0.7 0.9 0.8 0.2 0.7 0.6 0.8 1.  0.8 0.9]
[0.8 0.9 0.4 0.8 0.9 1.  1.  0.7 0.4 0.6]
[0.9 0.8 0.7 0.6 0.9 0.8 0.6 0.8 0.7 0.8]
[0.7 0.9 0.9 0.8 0.9 0.7 0.8 0.8 0.5 0.7]
[0.7 0.9 0.6 0.8 0.9 0.8 0.7 0.8 0.7 0.9]
Accuracy (mean, std): 0.78 0.10327955589886448
Precision (mean, std): 0.8013095238095238 0.10778387024882283
Recall (mean, std): 0.7823809523809524 0.10778387024882283
Area under the Receiver Operating Characteristic Curve 0.7823809523809524 0.10076677556965372
F1-score: 0.8009235209235209 0.12468386436825143


# Using mFDA>20 as label

In [0]:
# Import mFDA scores


mFDApath = '/content/drive/My Drive/2019 DDK Paper/Journal/Source Code/EMD Classification/ClassifyEMDdeltaEMD/'
mFDAPDaddr = glob.glob(mFDApath+'*PD.csv')
mFDAHCaddr = glob.glob(mFDApath+'*HC.csv')

print(mFDAPDaddr)
print(mFDAHCaddr)


dfPDmFDA = pd.read_csv(mFDAPDaddr[0])
dfPDmFDA=dfPDmFDA.drop(['code'], axis=1)
dfHCmFDA = pd.read_csv(mFDAHCaddr[0])
dfHCmFDA=dfHCmFDA.drop(['Code'], axis=1)
print(dfPDmFDA.head())
print(dfHCmFDA.head())


dfPDmFDA['label'] = [1 if mfda>20 else 0 for mfda in dfPDmFDA['mFDA'] ]
print('PD', dfPDmFDA)
dfHCmFDA['label'] = [1 if mfda>20 else 0 for mfda in dfHCmFDA['mFDA'] ]
print('HC', dfHCmFDA)



In [34]:
# Importing data matrices and remove rows containing nan
HCaddr = [addr for addr in addrs if 'HC' in addr ]
PDaddr = [addr for addr in addrs if 'PD' in addr ]

dfHCOriginal = pd.read_csv(HCaddr[0])
dfHCOriginal.dropna()
print('HC matrix dim: ', dfHCOriginal.shape)

dfPDOriginal = pd.read_csv(PDaddr[0])
dfPDOriginal.dropna()
print('PD matrix dim: ',dfPDOriginal.shape)


# Remove rows that contain 0 for all std features
def removeZeroData(dftmp):
  colnames = dftmp.columns
  stdcol = [header for header in colnames if 'Std' in header]
  stdtotal = dftmp[stdcol].sum(axis=1)
  dropIdx = [i for i,std in enumerate(stdtotal) if std==0]
  #print(stdtotal)
  print('Removing ', len(dropIdx), ' rows')
  for i in dropIdx:
    dftmp = dftmp.drop(dftmp.index[i])
  return dftmp

dfPDOriginal = removeZeroData(dfPDOriginal)
print(dfPDOriginal.shape)

dfHCOriginal = removeZeroData(dfHCOriginal)
print(dfPDOriginal.dtypes)    


HC matrix dim:  (50, 434)
PD matrix dim:  (50, 434)
Removing  0  rows
(50, 434)
Removing  0  rows
subject                int64
segmentNum             int64
numIMFMean           float64
OBW1Mean             float64
OBW2Mean             float64
                      ...   
deltaFCenter6Kurt    float64
deltaFCenter7Kurt    float64
deltaFCenter8Kurt    float64
deltaAmpKurt         float64
deltaDurKurt         float64
Length: 434, dtype: object


In [22]:
# Joining label with the rest of the dataframe, so the label can be aline with the subject
dfPD = pd.merge(dfPDOriginal, dfPDmFDA, on='subject')
dfHC = pd.merge(dfHCOriginal, dfHCmFDA, on='subject')


dfHCPD = pd.concat([dfHC,dfPD], axis=0)
print(dfHCPD.dtypes)
print(dfHCPD.head)


# Creating dataframe for classification
Labels = dfHCPD['label']
df = dfHCPD.drop(['subject','mFDA','label'], axis=1)
print(df)
print(Labels)



subject                int64
segmentNum             int64
numIMFMean           float64
OBW1Mean             float64
OBW2Mean             float64
                      ...   
deltaFCenter8Kurt    float64
deltaAmpKurt         float64
deltaDurKurt         float64
mFDA                 float64
label                  int64
Length: 436, dtype: object
<bound method NDFrame.head of     subject  segmentNum  numIMFMean  ...  deltaDurKurt       mFDA  label
0         1           7   17.857143  ...      0.899006   7.000000      0
1         3          14   18.000000  ...      2.211619   6.666667      0
2         4           8   18.375000  ...     -1.302787   4.000000      0
3         5           5   18.400000  ...     -1.178321  13.000000      0
4         6          11   17.818182  ...      0.809568  13.000000      0
..      ...         ...         ...  ...           ...        ...    ...
45       55          14   17.214286  ...      1.296257  33.000000      1
46       56           3   18.000000  ...

In [24]:
dffeat = df.values

# Normalizing the features into [0,1]
scaler = MinMaxScaler(feature_range = (0,1)) # scale the values to min 0, max 1
rescaleddffeat = np.array(scaler.fit_transform(dffeat)) # fit the trainning feature X into scaler

set_printoptions(precision=3) # how many decimal places.
print(rescaleddffeat[0:5,:])
print(rescaleddffeat.shape)

[[0.278 0.429 0.499 ... 0.881 0.193 0.447]
 [0.667 0.467 0.592 ... 0.607 0.379 0.649]
 [0.333 0.567 0.    ... 0.399 0.381 0.107]
 [0.167 0.573 0.63  ... 0.391 0.095 0.127]
 [0.5   0.418 0.877 ... 0.598 0.196 0.433]]
(100, 433)


In [41]:
# Finding best results with mFDA>20 as label

Iterations = 200
num_folds = 10
print(Labels)

print('Support Vector Machine')
acc, pre, rec, rocauc, f1 = findbestresults(SVM, Iterations, num_folds,  rescaleddffeat, Labels.values)
print('Accuracy (mean, std):', statistics.mean(acc), statistics.stdev(acc))
print('Precision (mean, std):', statistics.mean(pre), statistics.stdev(pre))
print('Recall (mean, std):', statistics.mean(rec), statistics.stdev(pre))
print('Area under the Receiver Operating Characteristic Curve', statistics.mean(rocauc), statistics.stdev(rocauc))
print('F1-score:', statistics.mean(f1), statistics.stdev(f1))



0     0
1     0
2     0
3     0
4     0
     ..
45    1
46    1
47    0
48    1
49    1
Name: label, Length: 100, dtype: int64
Support Vector Machine
[0.5 0.9 0.7 0.8 0.8 0.4 0.6 0.6 0.4 0.8]
[0.8 0.7 0.5 0.9 0.9 0.7 0.6 0.9 0.6 0.3]
[0.8 0.7 0.5 1.  0.5 0.6 0.9 0.6 0.8 0.8]
[0.9 0.6 0.7 0.7 0.6 0.6 0.8 0.7 0.8 0.9]
Accuracy (mean, std): 0.73 0.1159501808728406
Precision (mean, std): 0.774702380952381 0.11530193651724092
Recall (mean, std): 0.7311904761904762 0.11530193651724092
Area under the Receiver Operating Characteristic Curve 0.7311904761904762 0.10004219316469586
F1-score: 0.6644733044733044 0.1326478948936109
